# TPB708 Programación de Aplicaciones en Sistemas de Información Geográfica
## Proyecto 02 - Visualización de tablas, gráficos y mapas

### Marcelo Cubero 
### Johan Córdoba 
### Marco Arce

In [54]:
from __future__ import print_function

import pandas as pd
import geopandas as gpd

import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from plotly.offline import iplot

import chart_studio.plotly as py

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import folium

In [55]:
# Carga en dataframes de casos positivos, activos, recuperados y fallecidos para todas las fechas
positivos_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_POSITIVOS.csv", 
                           encoding = "latin")
activos_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_ACTIVOS.csv", 
                           encoding = "latin")
recuperados_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_RECUP.csv", 
                           encoding = "latin")
fallecidos_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_FALLECIDOS.csv", 
                            encoding = "latin")

# Carga en un dataframe de casos positivos, activos, recuperados y fallecidos para la última fecha
ultima_fecha_df = pd.read_csv("https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/covid19/ministerio-salud/11_11_CSV_ULTIMA_FECHA.csv", 
                               encoding = "latin")

# Cantidades de casos (positivos, recuperados, activos y fallecidos) para los cantones de la provincia de Cartago, para la última fecha disponible

In [56]:
ultima_fecha_tabla = ff.create_table(ultima_fecha_df[ultima_fecha_df['provincia'] == 'Cartago'])
iplot(ultima_fecha_tabla)

#  Cantidades de casos positivos correspondientes a la última semana de octubre de 2020, para los cantones de la provincia de Heredia

In [57]:
positivos_df_1 = positivos_df[['provincia', 'canton', '25/10/2020', '26/10/2020', '27/10/2020' , '28/10/2020',
                       '29/10/2020', '30/10/2020', '31/10/2020']]
positivos_df_1
positivos_df_1.columns = positivos_df_1.columns.str.replace('/', '')
positivos_df_1.columns = positivos_df_1.columns.str.replace('2', 'a')
positivos_df_1.columns = positivos_df_1.columns.str.replace('3', 'a')
positivos_df_1.columns

fig = go.Figure(data=[go.Table(
    header=dict(values=['Cantón', 'Provincias', '25/oct/2020', '26/oct/2020', '27/oct/2020', '28/oct/2020', '29/oct/2020',
                         '30/oct/2020', '31/oct/2020'],
                fill_color='palegreen',
                align='center'),
    cells=dict(values=[positivos_df_1.canton[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.provincia[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.a510a0a0[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.a610a0a0[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.a710a0a0[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.a810a0a0[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.a910a0a0[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.a010a0a0[ultima_fecha_df['provincia'] == 'Heredia'],
                       positivos_df_1.a110a0a0[ultima_fecha_df['provincia'] == 'Heredia'],
                       
                      ],
               fill_color='gold',
               align='center'))
])

fig.show()

#  Gráfico de barras de las cantidades de casos positivos en los cantones de Costa Rica con mayor cantidad de casos.

In [58]:
ultima_fecha_ordenado_df = ultima_fecha_df.sort_values("positivos", ascending=False)

fig = px.bar(ultima_fecha_ordenado_df.head(10), 
                 x="canton",
                 y="positivos", 
                 labels = {'canton': 'Cantones', 'positivos': 'Número de casos Positivos'},
                 color="positivos", 
                 hover_name="canton", 
                 color_continuous_scale='sunsetdark',
                 title = 'Cantones con más casos en Costa Rica'
                 )

fig.show()

# Mapa de símbolos proporcionales,  cantidad de casos en cada cantón

In [59]:
datos = ultima_fecha_df[['provincia','canton','fecha','positivos','recuperados','fallecidos','activos']]

def casos_cantones_tipo(tipo_casos):
    cantones = gpd.read_file ('https://raw.githubusercontent.com/tpb708-programacionsig-2020/datos/main/delimitacion-territorial-administrativa/cr/ign/cr_limite_cantonal_ign_wgs84.geojson')
  

        
    gm = folium.Map(
    location=[9.65, -84],
    zoom_start=8, 
    tiles='openstreetmap',
    control_scale=True)
    
    tonos = "Spectral"
    
    if tipo_casos == "activos":
        colores = "Reds"
    elif tipo_casos == "recuperados":
        colores = "Greens"
    elif tipo_casos == "fallecidos":
        colores = "Greys"
    
    c= folium.Choropleth(
        geo_data=cantones,
        data=ultima_fecha_df,
        columns=['cod_canton', tipo_casos],
        key_on='feature.properties.cod_canton',
        fill_color=tonos, 
        fill_opacity=1, 
        line_opacity=1,
        legend_name='Casos ' + tipo_casos,
        smooth_factor=0).add_to(gm)

    
    for feature in c.geojson.data['features']:
        canton_filtrado_df = ultima_fecha_df[ultima_fecha_df['cod_canton'] == feature['properties']['cod_canton']]
        feature['properties']['texto_emergente'] = feature['properties']['canton'] + "<br>" + str(canton_filtrado_df.iloc[0][tipo_casos])
        valores = []

    
    c.geojson.add_child(folium.features.GeoJsonTooltip(['texto_emergente'], labels=False))
    
    def puntos_xy(pt):
        return (pt.x, pt.y)
    g_cantones = cantones['geometry'].centroid
    x,y = [list(t) for t in zip(*map(puntos_xy, g_cantones))]
    
    cod = list(cantones['cod_canton'])
    
    circulos_df = pd.DataFrame({
           'lon':x,
           'lat':y,
           'cod':cod,
    })
    
    info_total = ultima_fecha_df[['cod_canton','positivos','recuperados','fallecidos','activos']]
    
    for i in range(0,len(circulos_df)):
        info_canton = datos[ultima_fecha_df['cod_canton'] == cod[i]]
        cantidad = info_canton[tipo_casos]
        cantidad2 = cantidad
        if tipo_casos == 'fallecidos':
            cantidad2 = cantidad2 * 75
        folium.Circle(
            location=[circulos_df.iloc[i]['lat'], circulos_df.iloc[i]['lon']],
            popup= int(cantidad),
            radius= int(cantidad2),
            color= 'gold',
            fill=True,
            fill_color= 'black'
        ).add_to(gm)


   
    return gm

casos_cantones_tipo("fallecidos")

<ipython-input-59-3c5f0df3a675>:45: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


